In [1]:
import pandas
import re
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
estudiantes = [
    {
        "nombre": "Estudiante Bueno",
        "nota_promedio": 90,
        "asistencia": 0.95,
        "observaciones": [
            "Participativa y responsable en clase",
            "Colaboradora con compañeros",
            "Siempre motivada y proactiva"
        ]
    },
    {
        "nombre": "Estudiante Promedio",
        "nota_promedio": 75,
        "asistencia": 0.80,
        "observaciones": [
            "Se esfuerza, pero a veces desinterés",
            "Responsable con tareas, pero distraído",
            "Participa ocasionalmente"
        ]
    },
    {
        "nombre": "Estudiante Malo",
        "nota_promedio": 60,
        "asistencia": 0.65,
        "observaciones": [
            "Desinterés en clase",
            "Falta de motivación",
            "Problemas con compañeros y bajo rendimiento"
        ]
    }
]


In [3]:
# Dataset de ejemplo para entrenar el modelo
# etiqueta: 1=positivo, 0=negativo

# Dataset de ejemplo para entrenar el ML manual
dataset = [
    # ==== POSITIVAS (1) ====
    ("Estudiante participativo y atento en clase", 1),
    ("Demuestra motivación y esfuerzo constante", 1),
    ("Mantiene una actitud positiva y colaboradora", 1),
    ("Responsable con tareas y proyectos asignados", 1),
    ("Se integra bien con sus compañeros", 1),
    ("Muestra iniciativa y es proactivo", 1),
    ("Atención excelente y participación activa", 1),
    ("Colabora con el equipo y respeta normas", 1),
    ("Demuestra interés genuino por aprender", 1),
    ("Actitud ejemplar y comprometida", 1),
    ("Asistencia perfecta y puntualidad constante", 1),
    ("Plantea preguntas profundas y busca aclaraciones", 1),
    ("Toma apuntes organizados y sigue el hilo de la clase", 1),
    ("Ayuda espontáneamente a compañeros con dificultades", 1),
    ("Busca material adicional para complementar lo aprendido", 1),
    ("Muestra curiosidad intelectual y va más allá del mínimo", 1),
    ("Aplica conceptos aprendidos a situaciones nuevas", 1),
    ("Solicita retroalimentación para mejorar su desempeño", 1),
    ("Demuestra habilidades de liderazgo positivo en equipos", 1),
    ("Mantiene una comunicación respetuosa con profesores", 1),
    ("Organiza su tiempo eficientemente para cumplir plazos", 1),
    ("Reflexiona sobre sus errores y busca corregirlos", 1),
    ("Participa en actividades extracurriculares académicas", 1),
    ("Muestra perseverancia ante tareas complejas", 1),
    ("Trabaja de forma autónoma con excelentes resultados", 1),
    ("Expresa gratitud y reconocimiento hacia el aprendizaje", 1),
    ("Adapta sus estrategias según las necesidades del curso", 1),
    ("Demuestra empatía y apoyo emocional a compañeros", 1),
    ("Contribuye a crear un ambiente de clase productivo", 1),
    ("Muestra balance entre trabajo académico y bienestar", 1),

    # ==== NEUTRAS ====
    ("Cumple con lo requerido sin destacar", 1),
    ("Participa ocasionalmente", 1),
    ("Actitud estable, sin comportamientos problemáticos", 1),
    ("Desempeño aceptable en la mayoría de las actividades", 1),
    ("Cumple requisitos mínimos sin sobresalir ni fallar", 1),
    ("Participa solo cuando se le solicita directamente", 1),
    ("Desempeño fluctuante entre diferentes asignaturas", 1),
    ("Requiere recordatorios ocasionales para cumplir tareas", 1),
    ("Interés variable dependiendo del tema específico", 1),
    ("Prefiere trabajar individualmente antes que en equipo", 1),
    ("Sigue instrucciones pero sin aportar ideas propias", 1),
    ("Evita conflictos pero también evita involucrarse", 1),
    ("Muestra timidez que limita su participación activa", 1),
    ("Rendimiento aceptable pero con potencial desaprovechado", 1),
    ("Completa trabajos justo en la fecha límite", 1),
    ("Asiste regularmente pero con participación limitada", 1),
    ("Respeta normas sin generar impacto positivo adicional", 1),
    ("Aprende mejor con estructura clara y guía constante", 1),
    ("Necesita validación frecuente para continuar trabajando", 1),

    # ==== NEGATIVAS (0) ====
    ("Desinterés en las actividades de clase", 0),
    ("Falta de motivación evidente", 0),
    ("Se distrae con facilidad durante la clase", 0),
    ("Problemas frecuentes con compañeros", 0),
    ("Actitud conflictiva y poco colaboradora", 0),
    ("No entrega tareas y muestra poco compromiso", 0),
    ("Bajo rendimiento y poca participación", 0),
    ("Demuestra desorganización constante", 0),
    ("Dificultades para trabajar en grupo", 0),
    ("Desmotivación y apatía ante el aprendizaje", 0),
    ("Ausencias frecuentes sin justificación adecuada", 0),
    ("Llegadas tardías recurrentes interrumpiendo la clase", 0),
    ("Uso constante de celular en actividades académicas", 0),
    ("Respuestas cortantes o irrespetuosas a preguntas", 0),
    ("Evasión sistemática de trabajos grupales", 0),
    ("Copiado en evaluaciones o plagio en trabajos", 0),
    ("Actitud desafiante hacia figuras de autoridad", 0),
    ("Aislamiento social persistente durante actividades", 0),
    ("Fatiga o somnolencia constante en horarios de clase", 0),
    ("Negativa a participar incluso en actividades sencillas", 0),
    ("Expresiones de frustración o ira ante desafíos académicos", 0),
    ("Falta de materiales básicos de forma reiterada", 0),
    ("Incapacidad para seguir instrucciones secuenciales", 0),
    ("Comportamientos disruptivos que afectan al grupo", 0),
    ("Descuido evidente en la presentación de trabajos", 0),
    ("Promesas de mejora que no se materializan en acciones", 0),
    ("Justificaciones constantes para bajo rendimiento", 0),
    ("Resistencia activa a metodologías de enseñanza", 0),
    ("Dificultades graves de concentración sostenida", 0),
    ("Indicios de problemas personales que afectan lo académico", 0),
    ("Comunicación agresiva o pasivo-agresiva con pares", 0),
    ("Abandono prematuro de actividades sin permiso", 0),
    ("Falta de objetivos académicos o profesionales claros", 0),
    ("Priorización de actividades no académicas durante clases", 0),
    ("Historial de conflictos con múltiples profesores", 0),
    ("Respuestas evasivas cuando se pregunta sobre su avance", 0),
    ("Deterioro progresivo en calidad de trabajo entregado", 0),
    ("Ausencia de contacto visual en interacciones académicas", 0),
    ("Postura corporal cerrada y defensiva constantemente", 0),
    ("Muestra signos de estrés académico severo no manejado", 0),
]


palabras_positivas = [
    "participativa", "colaboradora", "responsable", "motivado", "proactivo", 
    "esfuerzo", "destacado", "excelente", "atento", "comprometido", "activo",
    "puntual", "asistencia", "constante", "iniciativa", "liderazgo", "respetuoso",
    "organizado", "autónomo", "perseverante", "curioso", "profundo", "reflexivo",
    "empático", "gratitud", "balance", "adaptable", "eficiente", "sobresaliente",
    "ejemplar", "dedicación", "entusiasta", "creativo", "analítico", "metódico",
    "disciplinado", "focalizado", "resiliente", "innovador", "colaborativo",
    "comunicativo", "propositivo", "visionario", "ordenado", "preciso", "detallista",
    "solidario", "paciente", "tolerante", "flexible", "autocrítico", "mejora",
    "evolución", "crecimiento", "superación", "exigente", "calidad", "excelencia",
    "potencial", "talento", "habilidad", "capacidad", "inteligente", "ágil",
    "rápido", "eficaz", "productivo", "constructivo", "positivo", "optimista",
    "alegre", "energético", "dinámico", "vital", "entregado", "compromiso",
    "vocación", "pasión", "interés", "dedicación", "constancia", "regularidad",
    "confiable", "confianza", "seguridad", "autoestima", "autonomía", "independiente",
    "maduro", "serio", "formal", "educado", "cortés", "amable", "gentil",
    "considerado", "atento", "detallista", "cuidadoso", "meticuloso", "perfeccionista",
    "exhaustivo", "completo", "integral", "holístico", "equilibrado", "armónico"
]
palabras_negativas = [
    "desinterés", "problemas", "conflicto", "desmotivado", "bajo rendimiento", 
    "distrae", "falta de motivación", "ausencia", "tardanza", "celular", "irrespetuoso",
    "evasión", "copia", "plagio", "desafiante", "aislamiento", "fatiga", "somnolencia",
    "negativa", "frustración", "ira", "falta materiales", "incapacidad", "disruptivo",
    "descuido", "promesas incumplidas", "justificaciones", "resistencia", "dificultad",
    "concentración", "problemas personales", "agresivo", "pasivo-agresivo", "abandono",
    "ausencia objetivos", "priorización equivocada", "historial conflictivo", "evasivo",
    "deterioro", "contacto visual", "postura cerrada", "estrés", "apatía", "indiferencia",
    "negligencia", "irresponsable", "inconstante", "inestable", "volátil", "impulsivo",
    "agresividad", "violencia", "acoso", "bullying", "marginación", "exclusión",
    "discriminación", "prejuicio", "intolerante", "inflexible", "rígido", "terco",
    "obstinado", "negación", "evasión", "engaño", "mentira", "fraude", "trampa",
    "chantaje", "manipulación", "egoísmo", "individualismo", "narcisismo", "arrogancia",
    "soberbia", "prepotencia", "autoritario", "dominante", "controlador", "posesivo",
    "celoso", "envidioso", "rencoroso", "vengativo", "resentimiento", "amargura",
    "pesimismo", "derrotista", "fatalista", "catastrófico", "ansiedad", "depresión",
    "desánimo", "desaliento", "desesperanza", "abandono", "deserción", "desistimiento",
    "renuncia", "fracaso", "reprobación", "suspensión", "expulsión", "sanción",
    "amonestación", "llamado atención", "advertencia", "alerta", "peligro", "riesgo",
    "vulnerabilidad", "fragilidad", "debilidad", "limitación", "deficiencia",
    "carencia", "necesidad", "dependencia", "inseguridad", "inestabilidad", "caos",
    "desorden", "confusión", "desorientación", "perdido", "desubicado", "inadaptado",
    "incomprendido", "solitario", "aislado", "marginado", "excluido", "rechazado",
    "ignorado", "invisible", "silenciado", "callado", "tímido", "retraído", "introvertido",
    "inhibido", "bloqueado", "paralizado", "estancado", "regresión", "retroceso",
    "decaimiento", "disminución", "reducción", "pérdida", "deterioro", "empeoramiento",
    "agravamiento", "complicación", "crisis", "emergencia", "urgencia", "prioritario",
    "intervención", "asistencia", "apoyo", "seguimiento", "monitoreo", "evaluación",
    "diagnóstico", "tratamiento", "terapia", "rehabilitación", "recuperación",
    "reinserción", "reincorporación", "reintegración", "normalización", "estabilización"
]

# Palabras de contexto que pueden modificar el significado
modificadores_intensidad = [
    "muy", "extremadamente", "totalmente", "completamente", "absolutamente",
    "realmente", "verdaderamente", "genuinamente", "profundamente", "intensamente",
    "levemente", "ligeramente", "moderadamente", "parcialmente", "ocasionalmente",
    "frecuentemente", "constantemente", "siempre", "nunca", "jamás", "rara vez",
    "a veces", "generalmente", "habitualmente", "usualmente", "típicamente"
]

# Expresiones de riesgo especificas para estudiantes
expresiones_riesgo_estudiantil = [
    "riesgo de deserción", "posible abandono", "en observación", "necesita apoyo",
    "requiere atención", "situación vulnerable", "contexto complicado", "factores de riesgo",
    "indicadores alarmantes", "señales de alerta", "comportamiento preocupante",
    "rendimiento decreciente", "motivación en caída", "interés disminuido",
    "participación reducida", "compromiso menguante", "esfuerzo decreciente",
    "asistencia irregular", "puntualidad deficiente", "organización pobre",
    "planificación ausente", "metas indefinidas", "objetivos claros",
    "visión futura", "proyección académica", "expectativas realistas",
    "autoeficacia baja", "autoconcepto académico", "autoestima académica",
    "pertenencia escolar", "vinculación institucional", "identidad estudiantil"
]

def extraer_features(texto):
    texto = texto.lower()
    features = {}
    for p in palabras_positivas:
        if re.search(r"\b" + re.escape(p) + r"\b", texto):
            features[f"pos_{p}"] = 1
        else:
            features[f"pos_{p}"] = 0
    
    for n in palabras_negativas:
        if re.search(r"\b" + re.escape(n) + r"\b", texto):
            features[f"neg_{n}"] = 1
        else:
            features[f"neg_{n}"] = 0
    
    for m in modificadores_intensidad:
        if m in texto:
            features[f"mod_{m}"] = 1
        else:
            features[f"mod_{m}"] = 0
    
    for exp in expresiones_riesgo_estudiantil:
        if exp in texto:
            features[f"risk_{exp}"] = 1
        else:
            features[f"risk_{exp}"] = 0

    return features

X = [extraer_features(texto) for texto, label in dataset]
y = [label for texto, label in dataset]

vec = DictVectorizer(sparse=False)
X_vec = vec.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.3, random_state=42)
clf = LogisticRegression()
clf.fit(X_train, y_train)

def puntaje_ambiente(texto):
    features = vec.transform([extraer_features(texto)])
    return clf.predict_proba(features)[0][1]  # probabilidad de clase positiva

def puntaje_ambiente_estudiante(observaciones):
    puntajes = [puntaje_ambiente(obs) for obs in observaciones]
    return sum(puntajes)/len(puntajes) if puntajes else 0.5




In [4]:
# Formula de riesgo de desercion Rd, no bulto manito

PESO_CUALITATIVO = 0.50 # el factor cualitativo es el ambiente (F)

def calcular_riesgo(A, N, F, peso_cualitativo=PESO_CUALITATIVO):
    riesgo_asistencia = 1 - A
    riesgo_nota = max(0, 75 - N)/100  #
    riesgo_final = (0.25 * riesgo_asistencia) + (0.25 * riesgo_nota) + (peso_cualitativo * (1 - F))
    return round(riesgo_final, 3)

In [5]:
# Calculamos riesgo para todos los estudiantes

for e in estudiantes:
    F = puntaje_ambiente_estudiante(e["observaciones"])
    e["F"] = F
    e["Rd"] = calcular_riesgo(e["asistencia"], e["nota_promedio"], F)

ranking = sorted(estudiantes, key=lambda x: x["Rd"], reverse=True)

print("=== Ranking de riesgo de desercion ===")
for id, e, in enumerate(ranking, 1):
    print(f"{id}. {e['nombre']}: Rd = {e['Rd']}, Nota: {e['nota_promedio']}, Asistencia: {e['asistencia']*100:.0f}%, Puntaje ambiente: {e['F']:.2f}")
    

=== Ranking de riesgo de desercion ===
1. Estudiante Malo: Rd = 0.425, Nota: 60, Asistencia: 65%, Puntaje ambiente: 0.40
2. Estudiante Promedio: Rd = 0.266, Nota: 75, Asistencia: 80%, Puntaje ambiente: 0.57
3. Estudiante Bueno: Rd = 0.213, Nota: 90, Asistencia: 95%, Puntaje ambiente: 0.60
